# Hakai Nutrient QA-QC 
---
# Start Notebook
## Install packages 
Google colab servers has already a few commonly used packages installed. To install those missing, which are specific toh Hakai let's run the following commands:

In [ ]:
!uv sync
%env LOG_LEVEL=ERROR

## Import the python packages needed

In [ ]:
# Let's load pandas for working with the data in table
import pandas as pd 
import numpy as np

# Let's load seaborn to plot the data
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# This to install the hakai api tool and be able to download some data from hakai's database
from hakai_api import Client

# Install ioos_qc which is used to qc data
from ioos_qc import qartod

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets, HBox, VBox
# import time

from hakai_qc.flags import flag_color_map, flag_qartod_to_hakai
from  hakai_qc.qc import qc_dataframe

# Define some project standards and get Hakai API credentials 

In [ ]:
# And the interesting variables to use from the Hakai database
nutrient_variables = ['no2_no3_um','sio2','po4']

pd.options.display.max_columns = 100
client = Client(auth_flow="desktop")

# Download data from the Hakai database
For more information regarding the Hakai API, go [here](https://github.com/HakaiInstitute/hakai-api).

You can find a list of all the data type endpoints [here](http://hakaiinstitute.github.io/hakai-api/#endpoints).

In [ ]:
# Let's retrieve the endpoint to retrieve nutrients data:
endpointUrl = '/eims/views/output/nutrients'
site_id = 'QU39'
site_id = '1015 OUTLET'
start_time = '2022-06-11'
# end_time = '2026-06-11'
# We'll retrieve data only associated with QU39 between January 1st 2019 to January 1st 2020
filterUrl = 'site_id={0}&collected>{1}&limit=-1'.format(
    site_id, start_time
)
# filterUrl = 'collected>{0}&limit=-1'.format(
#     start_time
# )

# Make a data request for sampling stations
url = f'{client.api_root}/{endpointUrl}?{filterUrl}'
response = client.get(url)
df = pd.DataFrame(response.json())
df_original = df.copy()
original_columns = df.columns
print(f'{len(df)} records downloaded')
df.head()

In [ ]:
# Let's convert the collected time to a datetime object variable called  time 
#  and extract the from those datetime objects the year and month 
df['time'] = pd.to_datetime(df['collected'],utc=True).dt.tz_localize(None)
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['dayoftheyear'] = pd.to_timedelta(df['time'].dt.dayofyear, unit='d')

# Define a depth variable which is: 
#   - pressure_transducer_depth (if available)
#   - OR line_out_depth
df['depth'] = df['pressure_transducer_depth'].fillna(df['line_out_depth'])

# Review Replicates
## Pool Standard Deviation


In [ ]:
 # Let's create a pooled standard deviation function
def pooled_standard_deviation(df_to_review,count_col='count',std_col='std'):
    # Keep only records that have replicates
    df_replicates = df_to_review[df_to_review[count_col]>1]
    upper =  df_replicates[count_col].sub(1).mul(df_replicates[std_col].pow(2)).sum()
    lower = df_replicates[count_col].sub(1).sum()
    return np.sqrt(upper/lower)

# Get pool standard deviation values from replicates
df_grouped = df.groupby(['site_id','line_out_depth','collected'])[['no2_no3_um','sio2','po4']].agg(['mean','std','count'])

# printresults
for var in nutrient_variables:
    pool_std = pooled_standard_deviation(df_grouped[var])
    print('{0} pool.std.: {1}'.format(var,pool_std))

In [ ]:
def pool_std(stds,counts):
    upper =np.sum(np.subtract(counts,1)*np.power(stds,2))
    lower = np.subtract(np.sum(counts),len(counts))
    return np.sqrt((upper/lower))

def pool_std(stds,counts):
    upper = (counts.sub(1).mul(stds.pow(2))).sum()
    lower = (counts.sum()-len(counts)).sum()
    return np.sqrt(upper/lower)

pool_std(pd.Series([4.222321,5.823036,5.249778]),pd.Series([20,75,35]))

In [ ]:
df_grouped

In [ ]:
#df_stats = df_grouped.swaplevel(-1,0,axis='columns')
#df_stats
#df_stats.loc[(df_stats['count']>1).all(axis='columns')].groupby(level=1).apply(lambda x: pool_std(x.std,x.count))

## Replicates Standard Deviation Distribution
Present the distribution of the standard deviations the replicate samples for each line_out_depth.

In [ ]:
replicates = df_grouped.stack(level=0).dropna(subset=['std']).sort_index(level=1)
replicates.index.names = ['site_id', 'line_out_depth', 'collected', 'nutrients']

fig = px.histogram(replicates.reset_index(),
              x='std',color='line_out_depth', hover_name='collected',
              facet_row='nutrients')
fig.update_layout(width=1400, height=1500)
fig.update_xaxes(matches=None)
fig.for_each_xaxis(lambda xaxis: xaxis.update(showticklabels=True)) 
fig.update_yaxes(matches=None)
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.show()

# Time series QARTOD tests
Review each depth time series for a station and run timeseries specific test on them. The configuarion dictionary below, list of each variables and depth ranges the tests that will be applied.


## Set QARTOD Tests Configuration

In [ ]:
nutrients_qc_configs= {
    "-5 < depth < 50": """
        contexts:
                -   window:
                        starting: 2010-01-01T00:00:00
                        ending: null
                    streams:
                        no2_no3_um:
                            qartod:
                                gross_range_test:
                                    suspect_span: [0, 36]
                                    fail_span: [0, 40]
                        po4:
                            qartod:
                                gross_range_test:
                                    suspect_span: [0, 3]
                                    fail_span: [0, 4]
                        sio2:
                            qartod:
                                gross_range_test:
                                    suspect_span: [0,80]
                                    fail_span: [0,100]
        """
    ,
    "50<=depth": """
        contexts:
             -  window:
                    starting: 2010-01-01T00:00:00
                    ending: null
                streams:
                    no2_no3_um:
                        qartod:
                            gross_range_test:
                                suspect_span: [0, 36]
                                fail_span: [0, 40]
                            spike_test:
                                suspect_threshold: 2
                                fail_threshold: 3
                                method: 'differential'
                    po4:
                        qartod:
                            gross_range_test:
                                suspect_span: [0, 3]
                                fail_span: [0, 4]
                            spike_test:
                                suspect_threshold: 0.2
                                fail_threshold: 0.4
                                method: 'differential'
                    sio2:
                        qartod:
                            gross_range_test:
                                suspect_span: [0,80]
                                fail_span: [0,100]
                            spike_test:
                                suspect_threshold: 8
                                fail_threshold: 12
                                method: 'differential'
        """
}

## Run QARTOD Tests on Depth TimeSeries


In [ ]:
# Run QARTOD tests
overwrite_existing_flags = True
#overwrite_existing_flags = False
df = qc_dataframe(df.drop(columns=[col for col in df.columns if 'qartod' in col]),configs=nutrients_qc_configs,groupby=['site_id','line_out_depth'])
print(df[['no2_no3_um','no2_no3_flag','po4','po4_flag','sio2','sio2_flag']])
df = df.set_index(['index']).sort_index()
df['year'] = df['year'].astype(int)
# aggregate flags
for var in ['no2_no3_um','po4','sio2']:
    agg_flag = f"{var}_qartod_aggregate"
    df.loc[:,agg_flag] = qartod.qartod_compare(df.filter(like=var+'_qartod_').fillna(9).astype(int).transpose().to_numpy())
    # Map to hakai convention and update empty flags
    var_flag = "no2_no3_flag" if var == 'no2_no3_um' else f"{var}_flag"
    if overwrite_existing_flags:
        df.loc[:,var_flag] = df[agg_flag].replace(flag_qartod_to_hakai)
    else:
        df.loc[:,var_flag] = df[var_flag].fillna(df[agg_flag].replace(flag_qartod_to_hakai))


# Apply lower than detection limit flag as BDL
df.loc[df['no2_no3_um']<0.036 ,'no2_no3_um_flag']='BDL'
df.loc[df['po4']<0.032 ,'po4_flag']='BDL'
df.loc[df['sio2']<0.1 ,'sio2_flag']='BDL'


## Review QARTOD Results

In [ ]:
df.columns

In [ ]:
# flag_color_map = {key:value['Color'] for key,value in review.flag_conventions['HAKAI'].items()}

line_out_depths = line_out_depth=df['line_out_depth'].drop_duplicates().sort_values().tolist()
line_out_depth_selector = widgets.SelectMultiple(
    options=line_out_depths,
    value=[line_out_depths[0]],
    description='line_out_depth',
)
line_out_depths[0]
def make_qartod_plot_review(var, line_out_depth, site):
    flag_vars = df.filter(regex='flag|qartod').columns.tolist()
    df_site = df[df['site_id']==site]
    fig = px.line(df_site[df_site['line_out_depth'].isin(line_out_depth)].sort_values(['time',f"{var}_qartod_aggregate"]),
                x='time',y=var,
                color=f"{var}_flag" if var!='no2_no3_um' else "no2_no3_flag", 
                symbol='quality_level',
                hover_data=['hakai_id']+flag_vars,
                color_discrete_map=flag_color_map)
    for trace in fig.data:
        if 'AV' not in trace.name:
            trace.mode = 'markers'

    fig.update_layout(
        height=600,
        width=1200,
    )
    return fig.show()
interact(make_qartod_plot_review, var=['sio2','po4','no2_no3_um'],line_out_depth=line_out_depth_selector, site=df['site_id'].sort_values().unique())

# Red field ratio

In [ ]:
def get_red_field_plot(var,slope_limit,max_depth):
  labels= {'sio2':f'SiO2 (uM)', 'po4':'PO4 (uM)', 'line_out_depth':'Bottle Target Depth (m)'}
  figs=px.scatter(df.query("line_out_depth<@max_depth"),
                x=var,
                y='no2_no3_um',
                color='line_out_depth', 
                hover_data=['hakai_id','date'], 
                template='simple_white',
                title=labels[var],
                labels=labels, 
                facet_col='year')

  for id, item in enumerate(figs.data):
    figs.add_trace(go.Scatter(x = [0, slope_limit[0]],
                              y =  [0, slope_limit[1]],
                              mode='lines',
                              line_color='red',
                              showlegend=False
                              ),
                  row=1,
                  col=id+1)
  return figs

get_red_field_plot('po4',[2.1875,35],100)

In [ ]:
get_red_field_plot('sio2',[32.8125,35],100)

# Review Interannual Variability
Let's compute the average value measured for each depth and the associated standard deviation.

## Compute the seasonal variability
### Development 
Generate for each line_out_depth a seasonal variability model



In [ ]:
# Generate a reference depth variable
df['reference_depth'] = df['line_out_depth']
df.loc[df['reference_depth']>230, 'reference_depth'] = 260 # Let's group all data below 230 in the same group
df = df.sort_values('reference_depth')
df['reference_depth'] = df['reference_depth'].astype(str)

In [ ]:
# Reseample data to a standard grid for each year starting on Jan 1st. Average values within the same grid
grid_window = '14D'
df_resampled = pd.DataFrame()
resampled = []
groupby = ['site_id','year','reference_depth']
for index,df_group in df.groupby(['site_id','year','reference_depth']):
    df_temp = df_group.resample(grid_window,on='time',origin=pd.to_datetime(f"{index[1]}-01-01 00:00:00")).mean(numeric_only=True)
    df_temp[groupby] = index
    resampled += [df_temp]

df_resampled = pd.concat(resampled).reset_index()
df_resampled['dayoftheyear'] = df_resampled['time'].dt.dayofyear

# For each similar day of the year compute the interannual variability
df_interannual = df_resampled.drop(columns=['time']).groupby(['site_id','reference_depth','dayoftheyear']).agg(['mean','std']).reset_index()

# Center each window pandas resample give start of the window
df_interannual['dayoftheyear'] = df_interannual['dayoftheyear'] + pd.to_timedelta(grid_window).days/2 

In [ ]:
# Generate Upper and Lower limits based on standard deviation
alpha = 2
for var in nutrient_variables:
    df_interannual[(var,'lower_limit')] = df_interannual[var]['mean'] - alpha*df_interannual[var]['std']
    df_interannual[(var,'upper_limit')] = df_interannual[var]['mean'] + alpha*df_interannual[var]['std']

In [ ]:

def plot_seasonal(reference_depth,year,site,variable):
    df_filtered  = df_interannual.loc[(df_interannual['reference_depth'] == reference_depth) & (df_interannual['site_id']==site)]
    df_data_filtered = df.loc[(df['reference_depth'] == reference_depth) & (df['year'] == year) & (df['site_id'] == site)].copy()
    df_data_filtered['dayoftheyear'] = df_data_filtered['dayoftheyear'].dt.days

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_filtered['dayoftheyear'],y=df_filtered[variable]['mean'],name='interannual mean',line_color='black'))
    fig.add_trace(go.Scatter(x=df_filtered['dayoftheyear'],y=df_filtered[variable]['upper_limit'],line_color='lightgrey',mode=None,name=f'interannual mean + {alpha}*STD'))
    fig.add_trace(go.Scatter(x=df_filtered['dayoftheyear'],y=df_filtered[variable]['lower_limit'],line_color='lightgrey',fill='tonexty',mode=None,name=f'interannual mean - {alpha}*STD'))
    fig_data = px.line(df_data_filtered.sort_values(['reference_depth','dayoftheyear']),
                       x='dayoftheyear',y=variable,
                       line_dash='year',
                       color='no2_no3_flag' if variable =='no2_no3_um' else f"{variable}_flag",
                       hover_data=['hakai_id'],
                       color_discrete_map=flag_color_map)
    
    for item in fig_data.data:
        # If AV add a line between markers
        if 'AV' in item['name']:
            item['mode'] = 'lines+markers'
        else:
            item['mode'] = 'markers'

        fig.add_trace(item)
        
    fig.update_yaxes(title=variable)
    fig.update_xaxes(title='Day Of The Year')
    fig.update_layout(width=1500,
                      title='Interannual variability')
    fig.show()

year_selector = widgets.SelectMultiple(
    options=df['year'].sort_values().unique(),
    value=[df['year'].sort_values().unique()[0]],
    description="Select Year(s)",
)
depth_selector = widgets.SelectMultiple(
    options=df['reference_depth'].unique(),
    value=[df['reference_depth'].unique()[0]],
    description="Select depths",
)
site_selector = widgets.SelectMultiple(
    options=df['site_id'].unique(),
    value=[df['site_id'].unique()[0]],
    description="Select Site(s)",
)
interact(plot_seasonal,
         reference_depth= df['reference_depth'].unique(),
         year=df['year'].sort_values().unique(),
         site=df['site_id'].sort_values().unique(),
         variable=nutrient_variables)

## Make Monthly Box Plots

In [ ]:
def make_boxplot_per_depth(var,reference_depth,site):
    # Show the seasonality of the data, averaged per month for all years
    #  and give the interannual box pot statistic
    fig = px.box(df.query(f"reference_depth == '{reference_depth}' and site_id == '{site}'"),
                 x='month',y=var,hover_data=['hakai_id'])
    fig.update_layout(width=1500)
    return fig.show()
    
interact(make_boxplot_per_depth,
         var=nutrient_variables, 
         reference_depth = df['reference_depth'].unique(),
         site=df['site_id'].sort_values().unique(),
        )

# Create Aggregated Suggested Flag


## Review Profiles

In [ ]:
# Plot profile
group_profile_by = 'date'
profile_list = widgets.SelectMultiple(
    options=df[group_profile_by].sort_values().drop_duplicates(),
    value=(df[group_profile_by].iloc[0],),
    description="Select Profiles",
    disabled=False,
)
def make_profile_plot(profiles):
    # Filter only selected profiles
    # df_selected = df[df['date'].isin(profiles)]
    df_temp = df[df['date'].isin(profiles)].sort_values(['site_id','date','depth'])
    sites = df_temp['site_id'].unique()
    fig = []
    # print(df_temp[['site_id','no2_no3_flag','sio2_flag','po4_flag'] + nutrient_variables ])
    for site in sites:
        for var in nutrient_variables:
            df_site = df_temp[df_temp['site_id']==site].sort_values(['site_id','date','depth'])
            fig.append(px.line(df_site,x=var,y='depth',color=f"{var}_flag" if var!= "no2_no3_um" else "no2_no3_flag",hover_data=['hakai_id'], color_discrete_map=flag_color_map))
    
    
    subfig = make_subplots(rows=len(sites), cols=3, row_titles=sites.tolist())
    kk=1
    i=1
    for item in fig:
        for subitem in item.data:
            if kk>1 or i>1:
                subitem.showlegend=False
            subfig.add_trace(subitem,row=i,col=kk)
            subfig.update_xaxes(title_text=nutrient_variables[kk-1], row=i, col=kk)
            kk += 1
            if kk>3:
                kk=1
        subfig.update_yaxes(title_text='Depth (m)', row=i, col=1)
        if kk==1:
            i+=1
    subfig.update_yaxes(autorange="reversed")
    subfig.update_xaxes(matches=None)
    subfig.update_traces(mode='markers+lines')
    subfig.update_layout(width=1500, height=400*len(df_temp['site_id'].unique()))
    return subfig

interact(make_profile_plot, profiles=profile_list)

## Review Timeseries

# Get List of Sample Flagged

In [ ]:
# Show me the data flagged
df.loc[df[['no2_no3_flag','sio2_flag','po4_flag']].isin(['SVC','SVD','BDL']).any(axis='columns')][['date','hakai_id','no2_no3_flag','sio2_flag','po4_flag']]

# Save Result to Hakai Portal Compatible Excel Format

In [ ]:
df_output = df[original_columns]

# Rename Columns to Match Portal Output
# Drop unchanged rows

###ATTENTION### The output has not yet been tested with the Hakai API.
df_output.to_excel('Hakai_Nutrient_Revision_{0}.xlsx'.format(pd.Timestamp.now().isoformat().replace(':','')))


# Report Figures

## Contour plot

In [ ]:
# Plot contourf interpolate linearly over the x axis and maximum over two NaN values
def get_contour(var, site):
    df_site = df[df['site_id']==site]
    df_pivot = pd.pivot_table(df_site,values=var,index='line_out_depth',columns='date',aggfunc='mean').interpolate(axis='index',limit=4).sort_index(axis=0).sort_index(axis=1).interpolate(axis='columns',limit=3)
    fig = go.Figure(data =
        go.Contour(z=df_pivot.values,x=df_pivot.columns,y=df_pivot.index.values,
                colorbar=dict(title=dict(text=var)),
                colorscale='RdYlGn',
                ncontours=10,
                contours_coloring='heatmap'
                #,connectgaps=True
                ))
    fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                    ticks='outside',showline=True)
    fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
    fig.update_layout(width=1500)
    fig.show()

interact(get_contour,var=nutrient_variables, site=df['site_id'].sort_values().unique())

## Scatter with colorbar

In [ ]:
## Scatter with colorbar
fig = px.scatter(df.dropna(subset=['no2_no3_um'],axis=0),x='time',y='depth',
                 color='no2_no3_um',color_continuous_scale='RdYlGn',
                 hover_name='hakai_id')
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside')
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()

In [ ]:
# Add scatter per depth
px.scatter(df.sort_values(['line_out_depth','time']),
           x='time',
           y='no2_no3_um',
           color='depth',
           color_continuous_scale=px.colors.sequential.YlOrRd,
           hover_data=['hakai_id'])